In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler
from dataset import kline_data, prepare_prediction_data
from models import rnn_lstm, rnn_gru
from PyEMD import CEEMDAN

In [2]:
if not os.path.exists('saved_models'):
    os.mkdir('saved_models')
if not os.path.exists('plots'):
    os.mkdir('plots')

In [3]:
btc = kline_data()
btc.tail()

Last saved data candle close time: 2021-08-04 00:00:00


,open_time,open,high,low,close,volume,close_time,quote_av,trades,tb_base_av,tb_quote_av,ignore
1443,2021-07-30 00:00:00,40018.49,42316.71,38313.23,42206.37,73602.784805,2021-07-31 00:00:00,2.939834e+09,1797012,37730.426862,1.509173e+09,0.0
1444,2021-07-31 00:00:00,42206.36,42448.00,41000.15,41461.83,44849.791012,2021-08-01 00:00:00,1.868286e+09,1565380,22844.245050,9.518879e+08,0.0
1445,2021-08-01 00:00:00,41461.84,42599.00,39422.01,39845.44,53953.186326,2021-08-02 00:00:00,2.227567e+09,1603964,25796.753795,1.066580e+09,0.0
1446,2021-08-02 00:00:00,39850.27,40480.01,38690.00,39147.82,50837.351954,2021-08-03 00:00:00,2.010928e+09,1162878,24558.005343,9.715967e+08,0.0
1447,2021-08-03 00:00:00,39146.86,39780.00,37642.03,38207.05,57117.435853,2021-08-04 00:00:00,2.195361e+09,1379871,27683.466598,1.064119e+09,0.0


In [4]:
data = np.array(btc['close'])[-1200:-200]
minmax = MinMaxScaler()
data_norm = np.squeeze(minmax.fit_transform(np.expand_dims(data, axis=-1)))
data_norm.shape

(1000,)

In [5]:
input_size = 20
x, y = prepare_prediction_data(data_norm, input_size=input_size)
n_samples = x.shape[0]
x.shape, y.shape

((980, 20), (980,))

In [6]:
x_train = np.expand_dims(x[:int(n_samples * 0.85), :], axis=-1)
x_test = np.expand_dims(x[int(n_samples * 0.85):, :], axis=-1)
y_train = y[:int(n_samples * 0.85)]
y_test = y[int(n_samples * 0.85):]

In [ ]:
model = rnn_gru(input_size=input_size)
ea = EarlyStopping(patience=20)
cp = ModelCheckpoint('saved_models/gru_close.h5')
cb = [ea, cp]
history = model.fit(x_train, y_train, batch_size=32, epochs=1000, callbacks=cb, validation_split=0.3)

Epoch 1/1000
19/19 [==============================] - 3s 35ms/step - loss: 0.0972 - val_loss: 0.0028
Epoch 2/1000
19/19 [==============================] - 0s 9ms/step - loss: 0.0102 - val_loss: 0.0194
Epoch 3/1000
19/19 [==============================] - 0s 10ms/step - loss: 0.0084 - val_loss: 0.0083
Epoch 4/1000
19/19 [==============================] - 0s 10ms/step - loss: 0.0054 - val_loss: 0.0049
Epoch 5/1000
19/19 [==============================] - 0s 13ms/step - loss: 0.0051 - val_loss: 0.0064
Epoch 6/1000
19/19 [==============================] - 0s 18ms/step - loss: 0.0049 - val_loss: 0.0050
Epoch 7/1000
19/19 [==============================] - 0s 15ms/step - loss: 0.0045 - val_loss: 0.0051
Epoch 8/1000
19/19 [==============================] - 0s 16ms/step - loss: 0.0042 - val_loss: 0.0042
Epoch 9/1000
19/19 [==============================] - 0s 17ms/step - loss: 0.0039 - val_loss: 0.0034
Epoch 10/1000
19/19 [==============================] - 0s 19ms/step - loss: 0.0034 - val_los

19/19 [==============================] - 0s 22ms/step - loss: 1.4992e-04 - val_loss: 1.6680e-04
Epoch 77/1000
19/19 [==============================] - 0s 24ms/step - loss: 1.6320e-04 - val_loss: 1.5769e-04
Epoch 78/1000
19/19 [==============================] - 1s 27ms/step - loss: 1.4534e-04 - val_loss: 1.7626e-04
Epoch 79/1000
19/19 [==============================] - 0s 24ms/step - loss: 1.4992e-04 - val_loss: 1.7402e-04
Epoch 80/1000
19/19 [==============================] - 0s 20ms/step - loss: 1.5226e-04 - val_loss: 1.8795e-04
Epoch 81/1000
19/19 [==============================] - 0s 22ms/step - loss: 1.4783e-04 - val_loss: 1.5236e-04
Epoch 82/1000
19/19 [==============================] - 0s 24ms/step - loss: 1.7223e-04 - val_loss: 2.0520e-04
Epoch 83/1000
19/19 [==============================] - 0s 25ms/step - loss: 1.8183e-04 - val_loss: 1.4967e-04
Epoch 84/1000
19/19 [==============================] - 1s 27ms/step - loss: 1.4820e-04 - val_loss: 2.4229e-04
Epoch 85/1000
19/19 [===

19/19 [==============================] - 0s 23ms/step - loss: 9.2854e-05 - val_loss: 1.0651e-04
Epoch 151/1000
19/19 [==============================] - 0s 21ms/step - loss: 9.5069e-05 - val_loss: 1.0666e-04
Epoch 152/1000
19/19 [==============================] - 0s 23ms/step - loss: 9.0363e-05 - val_loss: 1.3068e-04
Epoch 153/1000
19/19 [==============================] - 0s 25ms/step - loss: 1.0090e-04 - val_loss: 1.0931e-04
Epoch 154/1000
19/19 [==============================] - 1s 30ms/step - loss: 9.3414e-05 - val_loss: 1.7524e-04
Epoch 155/1000
19/19 [==============================] - 0s 24ms/step - loss: 1.0803e-04 - val_loss: 1.8303e-04
Epoch 156/1000
19/19 [==============================] - 0s 24ms/step - loss: 9.2833e-05 - val_loss: 1.3178e-04
Epoch 157/1000
19/19 [==============================] - 0s 22ms/step - loss: 9.3882e-05 - val_loss: 1.5134e-04
Epoch 158/1000
19/19 [==============================] - 0s 23ms/step - loss: 1.0202e-04 - val_loss: 1.2598e-04
Epoch 159/1000
1

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

In [ ]:
print(f'Test set MSE: {model.evaluate(x_test[:3, :], y_test[:3], verbose=0)}')

In [ ]:
y_test_pred = np.squeeze(model.predict(x_test))

In [ ]:
plt.plot(np.squeeze(minmax.inverse_transform(np.expand_dims(y_test, axis=-1))), label='True')
plt.plot(np.squeeze(minmax.inverse_transform(np.expand_dims(y_test_pred, axis=-1))), label='Predicted')
plt.legend()